Para la realización de la primera tarea se estará recreando los ejercicios propuestos en el vídeo tutorial de "Codificando Bits" encontrado en el siguiente link https://www.youtube.com/watch?v=0mOV4plF2Xo\

Función para leer el conjunto de datos: Un función que permita cargar los datos de entrenamiento y validación, junto a sus categorías correspondientes.

In [4]:
import numpy as np
import os #para interactuar con funcionalidades del sistema operativo
import gzip #comprime y descomprime archivos .zip


def cargarset(ruta, tipo='train'):
  ruta_categorias = os.path.join(ruta, '%s-labels-idx1-ubyte.gz' % tipo)
  ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' % tipo)

  with gzip.open(ruta_categorias, 'rb') as rutacate:
    etiquetas = np.frombuffer(rutacate.read(), dtype=np.uint8, offset=8)
    
  with gzip.open(ruta_imagenes, 'rb') as rutaimg:
    imagenes = np.frombuffer(rutaimg.read(), dtype=np.uint8, offset=16).reshape(len(etiquetas), 784)

  return imagenes, etiquetas

Al presionar Shift+Enter o el ícono de un triángulo al lado del bloque de código, esto correrá el bloque en una de las CPUs disponibles en Google Collab.

Ahora, vamos a montar el drive en el servidor remoto de Google Colab para trabajar con los datos en nuestro Drive.

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
ruta = 'gdrive/My Drive/TopicosIA/fashion_mnist_data'

X_train, Y_train = cargarset(ruta, tipo='train')
X_test, Y_test = cargarset(ruta, tipo='test')

Ahora, antes de crear el modelo, debemos garantizar que el set de entrenamiento tenga de tamaño un múltiplo de 128, es decir, se reajustarán los datos. 

También usaremos reshape para asegurarnos que los datos serán imágenes en escala de grises de 28 * 278 píxeles

In [7]:
X_train = X_train[0:59904,:]
X_test = X_test[0:9984,:]
Y_train = Y_train[0:59904]
Y_test = Y_test[0:9984]

X_train = np.reshape(X_train,(59904,28,28,1))
X_test = np.reshape(X_test,(9984,28,28,1))

Ahora, para empezar con el modelo, primero debemos importar TensorFlow en su segunda versión

In [8]:
%tensorflow_version 2.x 
import tensorflow as tf
print('Versión de TensorFlow: ' + tf.__version__)

Versión de TensorFlow: 2.8.0


Ahora, una vez con TensorFlow2, vamos a importar la libería Keras:

Keras es una biblioteca de Python minimalista para Deep Learning que puede funcionar sobre Theano o TensorFlow. Fue desarrollada con el objetivo de que los modelos de Deep Learning sean tan rápidos y fáciles tanto para la investigación como el desarrollo. https://unipython.com/introduccion-y-como-instalar-keras-anaconda/

Primero se crea un contenedor del modelo con sequential, y luego se agregan las tres capas convolucionales. La diferencia entre estas 3 capas son su cantidad de filtros.

In [9]:
modelo = tf.keras.models.Sequential()
modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
modelo.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu')) #64 filtros
modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
modelo.add(tf.keras.layers.Dropout(0.25))

modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
modelo.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu')) #128 filtros
modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
modelo.add(tf.keras.layers.Dropout(0.25))

modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
modelo.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu')) #256 filtros
modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
modelo.add(tf.keras.layers.Dropout(0.25))

Después de estas tres capas vamos a aplanar los datos y agregaremos una red neuronal 

In [10]:
modelo.add(tf.keras.layers.Flatten())
modelo.add(tf.keras.layers.Dense(256)) #inicio con 256 neuronas
modelo.add(tf.keras.layers.Activation('elu'))
modelo.add(tf.keras.layers.Dropout(0.5))
modelo.add(tf.keras.layers.Dense(10)) #salida con 10 neuronas
modelo.add(tf.keras.layers.Activation('softmax'))

Ahora, se puede imprimir la información del modelo en la pantalla, un summary

In [11]:
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 28, 28, 1)        4         
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 64)        1664      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 14, 14, 64)       256       
 hNormalization)                                                 
                                                        

Antes del entrenamiento vamos a definir el optimizador a usar, la funcion de error y la metrica de desempeño.

In [12]:
modelo.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Entrenamiento con CPU!

Al ser una red convolucional compleja que tiene Total params: 1,619,470
Trainable params: 1,619,084, el tiempo de procesamiento será muy lento, así que las iteraciones serán solo 2.

Queremos comparar el rendimiento entre CPU, GPU y TPU, así que usaremos timeit para almacenar el tiempo de ejecución de cada iteración.

In [ ]:
import timeit

def entrenamiento_cpu():
  with tf.device('/cpu:0'):
    modelo.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1) #set de entrenamiento, set de validación, tamaño del lote, número de iteraciones
  
  return None

cpu_time = timeit.timeit('entrenamiento_cpu()', number=1, setup='from __main__ import entrenamiento_cpu') #almacenando el tiempo

Epoch 1/2
468/468 [==============================] - 809s 2s/step - loss: 0.6795 - accuracy: 0.7832 - val_loss: 0.3936 - val_accuracy: 0.8604
Epoch 2/2
468/468 [==============================] - 845s 2s/step - loss: 0.3922 - accuracy: 0.8611 - val_loss: 0.3333 - val_accuracy: 0.8853


In [ ]:
print('Tiempo con CPU: ' + str(cpu_time) + ' segundos')

Tiempo con CPU: 1653.8483386400003 segundos


El entrenamiento con CPU tardó un total de 

Entrenamiento con GPU!
Entrenaremos el mismo modelo pero cambiando la arquitactura.


*   En Hardware acceleration seleccionaremos GPU, en el anterior entrenamiento teníamos seleccionada la opción "none".
*   Verificaremos la disponibilidad de la GPU asignada por Google Colab para verificar que la GPU esté disponible para el entrenamiento.

*   Para este paso no solo basta cambiar de CPU a GPU, también es bueno volver a "llamar" a tensorflow, o mejor aún, todo el código volverlo a correr a excepción del entrenamiento con CPU





In [13]:
nombre_gpu = tf.test.gpu_device_name()
if nombre_gpu != '/device:GPU:0':
  raise SystemError('GPU no encontrada')
print('GPU encontrada: {}'.format(nombre_gpu))

GPU encontrada: /device:GPU:0


Ahora, verificado esto, procederemos con el entrenamiento.

In [14]:
import timeit

def entrenamiento_gpu():
  with tf.device('/device:GPU:0'):
    modelo.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1) #set de entrenamiento, set de validación, tamaño del lote, número de iteraciones
  
  
  return None

gpu_time = timeit.timeit('entrenamiento_gpu()', number=1, setup='from __main__ import entrenamiento_gpu') #almacenando el tiempo usado

Epoch 1/2
468/468 [==============================] - 28s 39ms/step - loss: 0.7053 - accuracy: 0.7747 - val_loss: 0.3901 - val_accuracy: 0.8540
Epoch 2/2
468/468 [==============================] - 19s 40ms/step - loss: 0.3944 - accuracy: 0.8624 - val_loss: 0.3122 - val_accuracy: 0.8882


A diferencia del entrenamiento con la CPU, la GPU resultó ser mucho más óptima, pues fue mucho más rápida con un tiempo menor a un minuto, mucho menor que los 26 minutos que s enecesitaron con la CPU.

In [15]:
print('Tiempo con GPU: ' + str(gpu_time) + ' segundos')

Tiempo con GPU: 47.23649320699997 segundos


Por último, hagamos el entrenamiento con una TPU!


*   Debemos verificar que en el hardware accelerator esté seleccionado la TPU
*   Ahora detectaremos la TPU y además usaremos algunas funciones de tensorflow para conectarnos a esta, inicializarla y crear un objeto para así lograr crear y compilar el modelo.

*   Para el entrenamiento con la TPU también es necesario correr d enuevo todo a excepción de los entrenamientos con CPU y TPU.






In [13]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])  # Detectar TPU
  print('TPU encontrada ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: TPU no encontrada!')

TPU encontrada  ['10.68.78.98:8470']


Ahora podemos proceder a conectarnos a la TPU e inicializarla

In [14]:
tf.config.experimental_connect_to_cluster(tpu) #conectar
tf.tpu.experimental.initialize_tpu_system(tpu) #inicializar
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu) #crear el objeto para poder correr el modelo

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.68.78.98:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.68.78.98:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Copiamos las líneas del modelo pero dentro de un with 

In [15]:
with tpu_strategy.scope():
  modelo = tf.keras.models.Sequential()
  modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  modelo.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
  modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  modelo.add(tf.keras.layers.Dropout(0.25))

  modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  modelo.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
  modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  modelo.add(tf.keras.layers.Dropout(0.25))

  modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  modelo.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
  modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  modelo.add(tf.keras.layers.Dropout(0.25))

  modelo.add(tf.keras.layers.Flatten())
  modelo.add(tf.keras.layers.Dense(256))
  modelo.add(tf.keras.layers.Activation('elu'))
  modelo.add(tf.keras.layers.Dropout(0.5))
  modelo.add(tf.keras.layers.Dense(10))
  modelo.add(tf.keras.layers.Activation('softmax'))
  modelo.summary()

  modelo.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_3 (Batc  (None, 28, 28, 1)        4         
 hNormalization)                                                 
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 64)        1664      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 14, 14, 64)        0         
                                                                 
 batch_normalization_4 (Batc  (None, 14, 14, 64)       256       
 hNormalization)                                                 
                                                      

Ahora, para proceder con el entrenamiento,  convertiremos los datos a formato punto flotante de 32 bits, pues es el requerido por Google Colab para usar entrenamientos con TPUs.

In [16]:
import timeit

x_train = X_train.astype(np.float32)
y_train = Y_train.astype(np.float32)
x_test = X_test.astype(np.float32)
y_test = Y_test.astype(np.float32)

def entrenamiento_tpu():
  modelo.fit(x_train,y_train,validation_data=(x_test,y_test), batch_size=128, epochs=2, verbose=1)
  
  return None

tpu_time = timeit.timeit('entrenamiento_tpu()', number=1, setup='from __main__ import entrenamiento_tpu')

Epoch 1/2
468/468 [==============================] - 15s 21ms/step - loss: 0.6953 - accuracy: 0.7807 - val_loss: 0.3507 - val_accuracy: 0.8718
Epoch 2/2
468/468 [==============================] - 8s 18ms/step - loss: 0.3966 - accuracy: 0.8601 - val_loss: 0.3022 - val_accuracy: 0.8893


Por último vemos que el uso de una TPU disminuye significativamente el tiempo del entrenamiento, siendo este de solo 26 segundos, mucho más rápido que los dos anteriores entrenamientos. Con este ejemplo podemos ver cómo lo más óptimo es el uso de TPUs.

In [17]:
print('Tiempo con TPU: ' + str(tpu_time) + ' segundos')

Tiempo con TPU: 26.58704345299998 segundos
